# OPTaaS Scikit-learn Pipelines

Using the OPTaaS Python Client, you can optimize any scikit-learn pipeline. For each step or estimator in the pipeline, OPTaaS just needs to know what parameters to optimize and what constraints will apply to them.

Your pipeline can even include **optional** steps (such as feature selection), **choice** steps (such as choosing between a set of classifiers) and **nested** pipelines.

We have provided pre-defined parameters and constraints for some of the most widely used estimators, such as Random Forest and XGBoost. The example below demonstrates how to use them. See also our [tutorial on defining your own custom optimizable estimators](sklearn_custom.ipynb).

## Load your dataset

We will run a classification pipeline using the German Credit Data available [here](https://newonlinecourses.science.psu.edu/stat857/node/215/). The data contains 1000 rows, with 20 feature columns and 1 target column which includes 2 classes.

In [1]:
import pandas as pd

data = pd.read_csv('german_credit.csv')
features = data[data.columns.drop(['Creditability'])]
target = data['Creditability']

## Create your OptimizablePipeline

Our pipeline will include:

- An optional feature selection step using PCA

- A choice of classifier from: Random Forest, Extra Trees and Linear SVC

In [2]:
from mindfoundry.optaas.client.sklearn_pipelines.estimators.pca import PCA
from mindfoundry.optaas.client.sklearn_pipelines.estimators.ensemble import RandomForestClassifier, ExtraTreesClassifier
from mindfoundry.optaas.client.sklearn_pipelines.estimators.svc import LinearSVC
from mindfoundry.optaas.client.sklearn_pipelines.mixin import OptimizablePipeline, choice, optional_step

pipeline = OptimizablePipeline([
    ('feature_selection', optional_step(PCA())),
    ('classification', choice(
        RandomForestClassifier(),
        ExtraTreesClassifier(),
        LinearSVC()
    ))
])

## Connect to the OPTaaS server using your API Key

We now create a client, and connect to the web service that will perform our optimization. You will need to input your personal API key. Make sure you keep your key private and don't commit it to your version control system. 

In [3]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Create your Sklearn Task

We don't need to worry about specifying all the parameters and constraints - they are generated based on our OptimizablePipeline. Sometimes we will need to provide additional kwargs, e.g. `feature_count` which is required by PCA.

If we do need to optimize any additional parameters that are outside of our pipeline, we can include them in `additional_parameters` and `additional_constraints`.

In [4]:
from mindfoundry.optaas.client.parameter import IntParameter
from mindfoundry.optaas.client.constraint import Constraint

my_extra_param = IntParameter('extra', id='extra', minimum=0, maximum=10)
my_extra_constraint = Constraint(my_extra_param != 7)

task = client.create_sklearn_task(
    title='My Sklearn Task', 
    pipeline=pipeline,
    feature_count=len(features.columns),
    additional_parameters=[my_extra_param],
    additional_constraints=[my_extra_constraint],
)

display(task.parameters)
display(task.constraints)

[{'id': '3055515155592',
  'items': [{'id': 'feature_selection',
    'items': [{'id': 'feature_selection__n_components',
      'maximum': 20,
      'minimum': 1,
      'name': 'n_components',
      'type': 'integer'},
     {'default': False,
      'id': 'feature_selection__whiten',
      'name': 'whiten',
      'type': 'boolean'}],
    'name': 'feature_selection',
    'optional': True,
    'type': 'group'},
   {'choices': [{'id': 'classification__0',
      'items': [{'default': 'auto',
        'enum': ['auto', 'sqrt', 'log2'],
        'id': 'classification__0__max_features',
        'name': 'max_features',
        'type': 'categorical'},
       {'default': 2,
        'distribution': 'Uniform',
        'id': 'classification__0__min_samples_split',
        'maximum': 20,
        'minimum': 2,
        'name': 'min_samples_split',
        'type': 'integer'},
       {'default': 1,
        'id': 'classification__0__min_samples_leaf',
        'maximum': 20,
        'minimum': 1,
        'name

['#extra != 7']

## Generate the first pipeline

We get the first configuration from OPTaaS and use it to generate a pipeline:

In [5]:
configuration = task.generate_configurations()[0]
pipeline = task.make_pipeline(configuration)

display(pipeline)

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classification', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_lea...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

## Run your pipeline and calculate the result

We define a function to run our pipeline and calculate the mean score:

In [6]:
from sklearn.model_selection import cross_val_score

def get_result(pipeline):
    scores = cross_val_score(pipeline, features, target, scoring='f1_micro')
    mean_score = scores.mean()
    return mean_score

mean_score = get_result(pipeline)

display(f'Mean score: {mean_score:.3f}')

'Mean score: 0.706'

## Record the result in OPTaaS and generate the next pipeline

In [7]:
configuration = task.record_result(configuration, score=mean_score)
pipeline = task.make_pipeline(configuration)

## Repeat as necessary

In [8]:
number_of_iterations = 20

for _ in range(number_of_iterations):
    display("OPTaaS recommendation:", pipeline)
    mean_score = get_result(pipeline)
    display(f'Mean score: {mean_score:.3f}')
    configuration = task.record_result(configuration, score=mean_score)
    pipeline = task.make_pipeline(configuration)

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('classification', LinearSVC(C=0.21062529888941453, class_weight='balanced', dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.09512838099432841, verbose=0))])

'Mean score: 0.705'

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)), ('classification', ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=42, max_features='sqrt', max_leaf_n...,
           n_jobs=1, oob_score=False, random_state=None, verbose=0,
           warm_start=False))])

...

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)), ('classification', LinearSVC(C=0.8779242530560565, class_weight='balanced', dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.37696701731239435, verbose=0))])

'Mean score: 0.725'

'OPTaaS recommendation:'

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classification', LinearSVC(C=0.9399946144684908, class_weight='balanced', dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.46292784216128025, verbose=0))])

'Mean score: 0.707'

## Complete your task

In [9]:
task.complete()

best_result, best_configuration = task.get_best_result_and_configuration()
best_pipeline = task.make_pipeline(best_configuration)

display(best_result, best_pipeline)

{ 'configuration': '043a3f9b-a3cc-46df-8f0d-1921657f3bf6',
  'id': 3849,
  'score': 0.7289565014115912,
  'user_defined_data': None}

Pipeline(memory=None,
     steps=[('feature_selection', PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)), ('classification', LinearSVC(C=0.9497943437100475, class_weight='balanced', dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.46292784216128025, verbose=0))])